In [1]:
import pathlib

import requests
from bs4 import BeautifulSoup
from PIL import Image
from io import BytesIO

from fpdf import FPDF

import subprocess

import re

import datetime
import markdown2

import time

from selenium import webdriver

In [2]:
# def try_connection():
try: 
    requests.get("https://www.google.com/")
except Exception as e:
    print(e)
    input()

In [3]:
def load_image_form_url(url, filename=None):
    page = requests.get(url)
    img = Image.open(BytesIO(page.content))
    filename = "downloads/" + filename
    if filename:
        img.save(filename)
    return filename

In [4]:
%mkdir downloads

A subdirectory or file downloads already exists.


In [5]:
vyuzitelnost_png = load_image_form_url("http://flymet.meteopress.cz/vyuzit/0task.png", "vyuzitelnost.png")

In [6]:
kumuly_png = load_image_form_url("http://flymet.meteopress.cz/cr/cukh12.png", "kumuly.png")

In [7]:
synoptika_gif = load_image_form_url("http://portal.chmi.cz/files/portal/docs/meteo/om/evropa/analyza.gif", "synoptika.gif")

In [8]:
synoptika_pred_gif = load_image_form_url("http://portal.chmi.cz/files/portal/docs/meteo/om/evropa/preba/preba.gif",
                                     "synoptika_pred.gif")

In [9]:
meteogram_png = load_image_form_url("http://flymet.meteopress.cz/meteogram/PODHORANY.png", "meteogram.png");

In [10]:
def load_images_form_urls(url, image_names):
    image_names_data = []
    for image_name in image_names:
        name_data = load_image_form_url(f"http://flymet.meteopress.cz/cr/{image_name}", image_name)
        image_names_data.append(name_data)
    
    return image_names_data

In [11]:
# deficit konvektivni teploty
dkt_pngs = load_images_form_urls("http://flymet.meteopress.cz/cr/", ["cudf10.png", "cudf13.png", "cudf16.png"])

In [12]:
# vyskyt kumulu a vyska
cukh_pngs = load_images_form_urls("http://flymet.meteopress.cz/cr/", ["cukh10.png", "cukh13.png", "cukh16.png"])

In [13]:
# vyskyt kumulu a vyska
curyb_pngs = load_images_form_urls("http://flymet.meteopress.cz/cr/", ["curyb10.png", "curyb13.png", "curyb16.png"])

In [14]:
page = requests.get("http://meteo.rlp.cz/LKCV_meteo.htm")
metar_page = page.content
for line in metar_page.splitlines():
    line = str(line)
    if "METAR LKCV" in line:
        metar = re.findall(r'(METAR LKCV.*)=', line)[0]
        break
    if "SPECI COR LKCV" in line:
        metar = re.findall(r'(SPECI COR LKCV.*)=', line)[0]
        break
else:
    metar = "failed to retrieve METAR from http://meteo.rlp.cz/LKCV_meteo.htm"

print(metar)

try:
    wind_idx = metar.index("KT") - 5
    try:
        wind_dir = int(metar[wind_idx:wind_idx+3])
        print("wind direction: " + str(wind_dir))
    except ValueError:
        wind_dir = 0
        print("wind direction VRB")
except:
    wind_dir=0

if len(metar) > 70:
    delim = metar.find(" ", 60)
    metar = metar[:delim] + "\n" + metar[delim:]
metar

METAR LKCV 240700Z 26006KT 220V300 9999 FEW018 SCT110 13/08 Q1026 NOSIG RMK BLU BLU
wind direction: 260


'METAR LKCV 240700Z 26006KT 220V300 9999 FEW018 SCT110 13/08 Q1026\n NOSIG RMK BLU BLU'

In [15]:
prostejov = "http://portal.chmi.cz/files/portal/docs/meteo/oa/data_sondaz_prostejov//dsd500.png"
prostejov_link = "http://portal.chmi.cz/aktualni-situace/aktualni-stav-pocasi/" \
                 "ceska-republika/sondazni-mereni/sondazni-mereni-prostejov"
praha = "http://portal.chmi.cz/files/portal/docs/meteo/oa/data_sondaz//dsd500.png"
praha_link = "http://portal.chmi.cz/aktualni-situace/aktualni-stav-pocasi/" \
             "ceska-republika/sondazni-mereni/sondazni-mereni" 

from_east = 10 < wind_dir < 190

vystup_url = prostejov if from_east else praha
link = prostejov_link if from_east else praha_link
vystup_url_2 = praha if from_east else prostejov
link_2 = praha_link if from_east else prostejov_link

vystup_png = load_image_form_url(vystup_url, "vystup.png")
vystup_png_2 = load_image_form_url(vystup_url_2, "vystup2.png")

In [17]:
driver = webdriver.Chrome()

SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 77


In [16]:
try:
    driver = webdriver.Chrome()
except:
    pass

driver.get('http://portal.chmi.cz/predpovedi/predpovedi-pocasi/letecke/sportovni-letani')

sportovni = driver.find_element_by_xpath('//*[@id="loadedcontent"]/pre')

sportovni_predpoved = sportovni.text

# driver.close()

NameError: name 'driver' is not defined

In [ ]:
try:
#     driver = webdriver.Chrome()
    meteogram_lkpn_url = "http://portal.chmi.cz/files/portal/docs/meteo/ov/aladin/results/public/meteogramy" \
    "/meteogram_page_portal/m.html#Podho%C5%99any%20(LKPN)"

    driver.get(meteogram_lkpn_url)
    driver.implicitly_wait(4)

    meteogram_img_url = driver.find_element_by_xpath('//*[@id="met-img"]').find_element_by_tag_name("img").get_attribute("src")

    meteogram_img = load_image_form_url(meteogram_img_url, "meteogram_lkpn.png")

    driver.close()
except:
    meteogram_img = 'downloads/meteogram_lkpn.png'

In [ ]:
sportovni_predpoved = sportovni_predpoved.split("POZNAMKA - Odkazy")[0]

In [ ]:
def add_bold(line):
    if ":" not in line:
        return line
    split = line.split(":", maxsplit=1)
    return f"<B>{split[0]}:</B>{split[1]}"

fontsize = 12

sportovni_predpoved_html = "\n".join(add_bold(line) for line in sportovni_predpoved.splitlines())
sportovni_predpoved_html = markdown2.markdown(sportovni_predpoved_html).replace("\n", "</p><p>")
sportovni_predpoved_html = f'<font face="courier" size="{fontsize}">' + sportovni_predpoved_html + '</font>'

sportovni_predpoved_html = sportovni_predpoved_html.replace(r"<p><p>", r"<p>")
sportovni_predpoved_html = sportovni_predpoved_html.replace(r"</p></p>", r"</p>")
sportovni_predpoved_html = sportovni_predpoved_html.replace(r"</p><p></p><p>", r"</p><p>")
sportovni_predpoved_html = sportovni_predpoved_html.replace(r"<code>", r"")
sportovni_predpoved_html = sportovni_predpoved_html.replace(r"</code>", r"")
sportovni_predpoved_html = sportovni_predpoved_html.replace(r"<pre>", r"")
sportovni_predpoved_html = sportovni_predpoved_html.replace(r"</pre>", r"")

print(sportovni_predpoved_html)

In [ ]:
time = str(datetime.datetime.utcnow())[0:16]
hours = time[:13].replace(" ", "-")

print(time)

In [ ]:
from fpdf import html
class PocasiPdf(FPDF, html.HTMLMixin):
    def header(self):
        self.set_font('Courier', '', fontsize)
        self.cell(192, txt=f"vygenerovalo GLIDEINFO {time} UTC", align='R')
        
    def footer(self):
        self.set_font('Courier', '', fontsize)
        self.set_xy(10, 290)
        self.cell(192, txt=f"Martin Stránský stranma5@gmail.com", align='R')

In [ ]:
pdf = PocasiPdf()

pdf.add_page()
pdf.set_xy(10, 18)
pdf.set_font('Courier', '', 12)
pdf.multi_cell(w=192, h=4, txt=metar)
pdf.set_xy(10, 30)
pdf.set_font('Courier', '', 16)
pdf.cell(w=192, txt="CHMI SPORTOVNI PREDPOVED (link):", 
         link="http://portal.chmi.cz/predpovedi/predpovedi-pocasi/letecke/sportovni-letani")
pdf.set_xy(10, 32)
pdf.write_html(sportovni_predpoved_html)

pdf.add_page()
pdf.image(vystup_png, x=10, y=14, h=140, 
          link=link)
pdf.image(vystup_png_2, x=10, y=152, h=140, 
          link=link_2)

pdf.add_page()
pdf.image(meteogram_img, x=10, y=13, h=280, link=meteogram_lkpn_url)

pdf.add_page()
pdf.image(meteogram_png, x=10, y=13, h=140, link="http://flymet.meteopress.cz/meteogram/")
for i, dkt_png in enumerate(dkt_pngs):
    pdf.image(dkt_png, x=10+i*60, y=150, h=45, link="http://flymet.meteopress.cz/cr/?gmetyp=cudf&gmepic=16")
for i, cukh_png in enumerate(cukh_pngs):
    pdf.image(cukh_png, x=10+i*60, y=195, h=45, link="http://flymet.meteopress.cz/cr/?gmetyp=cukh&gmepic=16")
for i, curyb_png in enumerate(curyb_pngs):
    pdf.image(curyb_png, x=10+i*60, y=240, h=45, link="http://flymet.meteopress.cz/cr/?gmetyp=curyb&gmepic=16")

pdf.add_page()
pdf.image(synoptika_gif, x=10, y=13, w=192,
          link="http://portal.chmi.cz/aktualni-situace/aktualni-stav-pocasi/evropa/synopticka-situace")
pdf.image(synoptika_pred_gif, x=10, y=152, w=192,
          link="http://portal.chmi.cz/predpovedi/predpovedi-pocasi/evropa/synopticka-situace")

# pdf.image(kumuly_png, x=10, y=10, h=140, link="http://flymet.meteopress.cz/cr/?gmetyp=cukh&gmepic=12")
# pdf.image(vyuzitelnost_png, x=10, y=150, h=140, link="http://flymet.meteopress.cz/vyu1.php")

pocasi_pdf = pathlib.Path(f"pocasi_shrnuti_{hours}.pdf")

try:
    pdf.output(pocasi_pdf, "F")
    subprocess.run(['start', str(pocasi_pdf)], shell=True)
except:
    print("Zavři předchozí přehled nebo ho smaž. Restartuj počítač, pokud to nepomůže.")
    input()